# **Mask operations**

<div style="color:#777777;margin-top: -15px;">
<b>Author</b>: Norman Juchler |
<b>Course</b>: ADLS ISP |
<b>Version</b>: v1.2 <br><br>
<!-- Date: 06.05.2025 -->
<!-- Comments: Entirely refactored -->
</div>

In this notebook, we explore various operations involving image masks, along with useful techniques that can be applied in different image processing contexts.

---

## **Preparations**

The usual preparations... The package `isp` provides some helper functions to easily render images in this Jupyter notebook.

In [ ]:
import sys
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Enable vectorized output (for nicer plots)
%config InlineBackend.figure_formats = ["svg"]

# Inline backend configuration
%matplotlib inline

# Functionality related to this course
sys.path.append("..")
import isp

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

In this notebook, we will work with the following images. The first image shows a hematology sample (a blood smear) containing red blood cells and one white blood cell. The second image is a mask that labels only the red blood cells. 


In [ ]:
# The default dataset (feel free to change)
mask = cv.imread("../data/images/hematology-baso1-mask.png", cv.IMREAD_GRAYSCALE)
img = cv.imread("../data/images/hematology-baso1.jpg", cv.IMREAD_COLOR)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
isp.show_image_chain([img, mask], titles=["Image", "Mask (red blood cells)"], suppress_info=True, figsize=(9, 4))

For educational purposes, we introduce a second mask that has been distorted with pepper noise and randomly placed holes.

In [ ]:
# Distortion 1: Add Gaussian noise
np.random.seed(2)
mask_noisy = np.ones(mask.shape, dtype=np.uint8) * 255
mask_noisy[np.random.rand(*mask_noisy.shape) < 0.1] = 0

# Distortion 2: Add larger holes. Place the holes randomly, but at a 
# certain distance from the edges, so that the holes will not change the
# shape of the mask.
mask_central = cv.erode(mask, None, iterations=11)
indices = np.argwhere(mask_central > 0)
num_holes = 100
selection = np.random.choice(len(indices), num_holes, replace=False)
centers_holes = indices[selection]
mask_holes = np.ones(mask.shape, dtype=np.uint8)*255
for center in centers_holes:
    radius = np.random.randint(3, 9)
    # [::-1]: swap x and y coordinates
    cv.circle(mask_holes, tuple(center[::-1]), radius, 0, -1)
    
# Combine the masks
mask_distorted = cv.bitwise_and(mask_holes, mask)
mask_distorted = cv.bitwise_and(mask_distorted, mask_noisy)

isp.show_image_chain([mask, mask_distorted], 
                     titles=["Original mask", "Distorted mask"], 
                     suppress_info=True, figsize=(9, 4))


## **Connected components**

Assume you have a binary segmentation mask containing multiple objects. The goal is to identify the number of distinct objects and determine their bounding boxes. This brings us to the concept of connected components.

A **connected component** is a group of adjacent foreground pixels in a binary image that are connected based on a defined connectivity. Each connected component corresponds to a distinct object or region in the image.

In this context, **connectivity** defines how pixels are considered neighbors, see Figure 1. With 4-connectivity, only directly adjacent pixels (up, down, left, right) are connected. With 8-connectivity, diagonal neighbors are also included. The choice of connectivity influences how objects are separated or grouped during labeling.

<img src="../data/doc/connected-components-2d.png" style="width:40%;">

**Figure 1**: Pixel connectivity in 2D relative to the central pixel.

OpenCV provides a connected components algorithm that labels each connected region in a binary image. It returns a labeled image of the same size as the input, where each pixel is assigned a label indicating the connected component it belongs to. Background pixels are labeled with zero.

In [ ]:
# Connectivity: 4 or 8.
#   - 4: two pixels are connected if they share an edge.
#   - 8: two pixels are connected if they share an edge or a corner.
labels = cv.connectedComponents(mask, connectivity=8)[1]

The remainder of this section demonstrates different ways to visualize the individual connected components.

In [ ]:
# Container for visualization
results = {}
results["Input"] = mask

# Visualization 1: 
# ################
# Display the connected components in different colors.
def colorize_labels(labels):
    """
    Represent the data in HSV such that the connected components are colored
    differently. Recall: HSV refers to hue, saturation, and value. Hue 
    represents an angle in the color wheel, saturation the intensity of the 
    color, and value the brightness. To represent black, we set value to zero.
    """
    h = labels/labels.max()*128
    s = np.ones_like(labels)*255
    v = (labels > 0) * 255
    labels_color = np.stack([h, s, v], axis=-1)
    labels_color = cv.cvtColor(labels_color.astype(np.uint8),
                               cv.COLOR_HSV2RGB)
    return labels_color

labels_color = colorize_labels(labels)
results["Connected components"] = labels_color

# Visualization 2: 
# ################
# Restrict the number of colors and shuffle them. 
# We can use a look-up table (LUT) for this purpose.
def colorize_labels_random(labels):
    h_vals = [1, 34, 55, 99]  
    assert 0 not in h_vals  # Zero is reserved for the background
    # For reproducability
    np.random.seed(1)
    # Sample n times with replacement from h_vals 
    lut = np.random.choice(h_vals, labels.max())
    # Insert a zero at the beginning to represent the background
    lut = np.insert(lut, 0, 0)
    # Lookup values
    labels_new = lut[labels]
    labels_color = colorize_labels(labels_new)
    return labels_color

labels_color = colorize_labels_random(labels)
results["Randomized colors"] = labels_color

isp.show_image_grid(results, figsize=(10, 10), suppress_info=True)

The above code is powered by `cv.connectedComponents()`. Note that we can also use  `scipy.ndimage.label()` for this purpose. The latter is more flexible and can be used for n-dimensional data.

In [ ]:
import scipy.ndimage as si
labels, n_labels = si.label(mask)

# It's output is equivalent to cv.connectedComponents().
ret = colorize_labels(labels)

# Compare the results
isp.show_image_chain([results["Connected components"], ret],
                     titles=["OpenCV: cv.connectedComponents()", 
                             "Scipy: scipy.ndimage.label()"], 
                     suppress_info=True)

We may want to compute various properties for each of the extracted components. Before doing so, let's introduce a few related concepts—such as contours and convex hulls—that will be useful for this purpose.

## **Contours**

Contours are the boundaries of connected components in a binary image. They can be extracted directly using `cv.findContours()`.

This function returns two values: a list of contours and hierarchy information (as a list). The hierarchy contains four indices for each contour: `[next, previous, first child, parent]`. Each value is an index referring to another contour; if no such relation exists, the value is -1.

In the following examples, we will ignore the hierarchy information. However, it becomes useful when dealing with nested contours – such as objects that contain holes or are enclosed within other objects.

The `mode` argument in `cv.findContours()` determines how contours are retrieved. The most common ones are:
- `cv.RETR_EXTERNAL`: Retrieve only the outermost (external) contours.
- `cv.RETR_LIST`: Retrieve all contours in a flat list, without hierarchical information.
- `cv.RETR_CCOMP`: Retrieve all contours as a two-level hierarchy (external and holes).
- `cv.RETR_TREE`: Retrieve all contours and reconstructs the full hierarchy as a tree.

The `method` parameter controls how the contour points are approximated. The goal is to reduce the number of points while preserving the overall shape of the contour.


In [ ]:
contours, _ = cv.findContours(mask, 
                              mode=cv.RETR_EXTERNAL, 
                              method=cv.CHAIN_APPROX_SIMPLE)

The `cv.drawContours()` function allows us to conveniently draw and fill contours. In the following examples, we demonstrate several ways to use this function.

In [ ]:
results = {}
results["Input"] = mask

# Visualization 1: Simple contours
# ################################
img_outlines = np.zeros_like(img)
color = [255, 255, 0]
cv.drawContours(img_outlines, contours, 
                contourIdx=-1, 
                color=color, 
                thickness=2)
results["Plain contours"] = img_outlines

# Visualization 2: Single contour (filled)
# ########################################
img_outlines = np.zeros_like(img)
color = [50, 100, 255]
contour_id = 42
cv.drawContours(img_outlines, contours, 
                contourIdx=contour_id, 
                color=color, 
                thickness=-1)
results["Single contour (filled)"] = img_outlines

# Visualization 3: Use colors
# ###########################
import matplotlib as mpl
# Use colormaps from matplotlib
# https://matplotlib.org/stable/users/explain/colors/colormaps.html
cmap = mpl.colormaps["inferno"]
colors = [cmap(i) for i in np.linspace(0, 1, len(contours))]
# Convert colors from [0, 1] to [0, 255]
colors = [[int(c*255) for c in color] for color in colors]
img_outlines = img.copy()
for i, contour in enumerate(contours):
    cv.drawContours(img_outlines, 
                    contours, 
                    contourIdx=i, 
                    color=colors[i], 
                    thickness=3)
results["Contours in color"] = img_outlines

# Visualization 4: Random colors
# ##############################
img_outlines = img.copy()
contours_shuffled = list(contours)
#np.random.shuffle(contours_shuffled)
colors = [[50, 100, 255],
          [255, 100, 50],
          [50, 255, 100]]
for i, contour in enumerate(contours_shuffled):
    cv.drawContours(img_outlines, 
                    contours_shuffled, 
                    contourIdx=i, 
                    color=colors[i % len(colors)], 
                    thickness=3)
results["Randomized colors"] = img_outlines

# Visualization 5: Colorize based on size
# #######################################
sizes = np.array([cv.contourArea(contour) for contour in contours])
sizes = sizes/sizes.max()  # Normalized sizes
img_outlines = img.copy()
# Use the following line if you want to sort the contours by size
# contours_sorted = sorted(contours, key=cv.contourArea)

# Use the colormap "hsv". It contains a transition red-yellow-green.
# cmap is a function that maps a scalar between 0 and 1 to a color.
# To sample only colors in the red-yellow-green range, we can use only
# the first 30% of the colormap. 
cmap = mpl.colormaps["hsv"]
colors = [cmap(s*0.3) for s in sizes]
colors = [[int(c*255) for c in color] for color in colors]

for i, contour in enumerate(contours):
    cv.drawContours(img_outlines, 
                    contours, 
                    contourIdx=i, 
                    color=colors[i % len(colors)], 
                    thickness=-1)  # Filled
results["Colors sorted by size"] = img_outlines

# Visualization 6: Bounding boxes and centroids
# #############################################
def draw_bounding_boxes(img, contours):
    assert img.ndim == 3
    img = img.copy()
    cmap = mpl.colormaps["hsv"]
    colors = [cmap(i) for i in np.linspace(0, 1, len(contours))]
    colors = [[int(c*255) for c in color] for color in colors]
    for i, contour in enumerate(contours):
        x, y, w, h = cv.boundingRect(contour)
        cv.rectangle(img, (x, y), (x+w, y+h), 
                    color=colors[i], thickness=2)
        
        # For the center of mass, we need to compute the moments of the contour
        # https://docs.opencv.org/4.x/dd/d49/tutorial_py_contour_features.html
        M = cv.moments(contour)
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        cv.circle(img, (cx, cy), 5, color=colors[i], thickness=-1)
    return img

img_in = cv.cvtColor(mask, cv.COLOR_GRAY2RGB)
ret = draw_bounding_boxes(img_in, contours)
results["Bounding boxes"] = ret

ret = draw_bounding_boxes(img, contours)
results["Bounding boxes (as overlay)"] = ret

isp.show_image_grid(results, figsize=(10, 9), suppress_info=True)

## **Measuring components**
In the following, we demonstrate how to compute various metrics – such as area, perimeter, and circularity – for each individual component (in this case, each red blood cell).

We use the function `cv.connectedComponentsWithStats()` for this purpose. It returns four values:
1. `num_labels`: The total number of connected components found (including the background).
2. `labels`: A labeled image where each pixel value indicates the component it belongs to.
3. `stats`: An array where each row contains statistics for one component, including   
the bounding box (x, y, width, height) and the area
4. `centroids`: The (x, y) coordinates of the centroid (center of mass) for each component.


In [ ]:
ret = cv.connectedComponentsWithStats(mask, connectivity=8)
num_labels, labels, stats, centroids = ret

df = pd.DataFrame(stats, columns=["x", "y", "width", "height", "area"])
df["centroid_x"] = centroids[:, 0]
df["centroid_y"] = centroids[:, 1]
df["label"] = np.arange(num_labels)
df["label"] = df["label"].astype(np.uint8)

display(df.head(20))

## **Convex hull**

A convex object is a set of points such that the line segment connecting *any two points* in the set lies entirely within the set. See below for examples of convex and non-convex shapes.


<img src="../data/doc/convexity.svg" style="width:60%;">  

**Figure**: Illustration of non-convex (left) and convex shapes (middle and right). Source: [Link](https://d2l.ai/index.html)

The **convex hull** of a geometric object (or a set of points) is the smallest convex shape that fully encloses the object. It can be computed in OpenCV using the `cv.convexHull()` function.


In [ ]:
# Compute the convex hull for a binary mask
mask_text = cv.imread("../data/images/word-ice-cream.png", cv.IMREAD_GRAYSCALE)
mask_text = 255 - mask_text  # Invert the mask
contours_text, _ = cv.findContours(mask_text, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

drawing = cv.cvtColor(mask_text, cv.COLOR_GRAY2RGB)
color = (255, 0, 0)
for i in range(len(contours_text)):
    hull = cv.convexHull(contours_text[i])
    cv.drawContours(drawing, [hull], 0, color=color, thickness=1)

isp.show_image_chain([mask_text, drawing], ncols=1,
                     titles=["Input", "Convex hull"], 
                     suppress_info=True, 
                     figsize=(6, 5))

For our initial dataset, the convex hulls look as follows:

In [ ]:
# Compute the convex hull for a binary mask
drawing = img.copy()
color = (255, 255, 0)
for i in range(len(contours)):
    hull = cv.convexHull(contours[i])
    cv.drawContours(drawing, [hull], 0, color=color, thickness=2)

isp.show_image_chain([mask, drawing], 
                     titles=["Input", "Convex hull"], 
                     suppress_info=True, 
                     figsize=(8, 5))

## **Distance transform**

The distance transform computes the distance from each foreground pixel to the nearest background (zero-valued) pixel.  It is typically applied to binary images and produces a grayscale image where pixel values represent distances. This transformation is useful in tasks such as shape analysis, object separation, and skeletonization.

In [ ]:
results = {}
results["Input 1"] = mask
results["Input 2"] = mask_text

# Compute the distance transform
dist_transform = cv.distanceTransform(mask, cv.DIST_L2, cv.DIST_MASK_PRECISE)
dist_transform /= dist_transform.max()
results["Distance transform 1"] = dist_transform

dist_transform = cv.distanceTransform(mask_text, cv.DIST_L2, cv.DIST_MASK_PRECISE)
dist_transform /= dist_transform.max()
results["Distance transform 2"] = dist_transform

isp.show_image_grid(results, ncols=2, figsize=(10, 5), 
                    suppress_info=True, shape=None)


## **Hole filling**


Hole filling is a common operation used to close internal gaps or holes within objects in a binary image. In this context, holes are background pixels that are completely enclosed by foreground pixels. OpenCV provides several approaches for hole filling, two of which are demonstrated here.

Note that the first method, which is based on morphological operations, is effective for closing relatively small gaps. The second method, which relies on contours, is also capable of filling larger holes.

In [ ]:
# Method 1: Morphological operations (only works for small holes)
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
mask_filled1 = cv.morphologyEx(mask_distorted, cv.MORPH_CLOSE, kernel, iterations=2)

# Method 2: Contour filling
mask_filled2 = mask_distorted.copy()
contour, hierarchy = cv.findContours(mask_distorted, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)
for cnt in contour:
    cv.drawContours(mask_filled2,[cnt],0,255,-1)

# Visalize the results
isp.show_image_chain([mask_distorted, mask_filled1, mask_filled2], 
                     titles=["Noisy mask (input)", "Filled (morphology)", "Filled (contour)"],
                     figsize=(9, 5),
                     suppress_info=True)

## **Flood fill**

Suppose we want to fill a region in the mask with a specific gray level (e.g., 255), starting from a given pixel—but only if that pixel is connected to the top-left corner. This can be achieved using the flood fill algorithm.

In [ ]:
seed_point = (30, 30)

# Flood fill
result1 = np.zeros_like(mask_distorted, dtype=np.uint8)
mask_constraints = cv.copyMakeBorder(mask_distorted, 1, 1, 1, 1,
                                     cv.BORDER_CONSTANT, value=0)
cv.floodFill(result1, mask_constraints, seed_point, 255)

# Visualize the seed point as a red dot
mask_filled_with_seed = cv.cvtColor(result1, cv.COLOR_GRAY2RGB)
cv.circle(mask_filled_with_seed, seed_point, 7, [255,0,0], -1)
isp.show_image_chain([mask_distorted, mask_filled_with_seed, None], 
                      titles=["Input", "Filled mask (flood fill)", None],
                      suppress_info=True)

In [ ]:
# One can use a second mask to restrict the flood fill.
result2 = np.zeros_like(mask_distorted, dtype=np.uint8)

mask_constraints = cv.copyMakeBorder(mask_distorted, 1, 1, 1, 1,
                                     cv.BORDER_CONSTANT, value=0)
lw = 10  # Line width
mask_constraints = cv.circle(mask_constraints, (150, 150), 125, 255, lw)
mask_constraints = cv.rectangle(mask_constraints, (250, 100), (450, 300), 255, lw)
mask_constraints = cv.ellipse(mask_constraints, (500, 330), (100, 80), 30, 0, 360, 255, lw)
cv.floodFill(result2, mask_constraints, (0, 0), 255)

# Visualize the seed point as a red dot
mask_filled_with_seed = cv.cvtColor(result2, cv.COLOR_GRAY2RGB)
cv.circle(mask_filled_with_seed, seed_point, 7, [255,0,0], -1)
isp.show_image_chain([mask_distorted, mask_constraints, mask_filled_with_seed], 
                      titles=["Input", "Mask", "Filled mask (flood fill)"],
                      suppress_info=True)

Note that in the previous example, the flood fill is constrained not only by the mask but also by the image content. As a result, it does not fill the entire image. Only the pixels that are connected to the seed point (= have the same value) are filled.

## **Thinning**
Thinning (or skeletonization) is the process of reducing a binary image to a
skeleton representation. The skeleton is a thin representation of the object
that is useful for shape analysis. It can be computed using the Zhang-Suen
algorithm, which is implemented in the opencv-contrib-python package.

In [ ]:
# Compute the skeleton (requires the opencv-contrib-python package)
if (not hasattr(cv, "ximgproc")) or (not hasattr(cv.ximgproc, "thinning")):
    raise RuntimeError("This version of OpenCV does not support thinning.")

thinned = cv.ximgproc.thinning(mask_text)
ret = cv.cvtColor(mask_text, cv.COLOR_GRAY2RGB)
ret[thinned == 255] = [31, 41, 255]

isp.show_image_chain([mask_text, ret], ncols=1, 
                     titles=["Input (inverted mask)", 
                             "Skeletonization of the mask"], 
                     suppress_info=True, figsize=(6, 5))

Thinning can be used not only to reduce a binary image to its skeleton but also to identify paths that are equidistant from the object boundaries. An example is shown below. Note that in this case, the thinning is applied to the background (i.e., the inverted mask).

In [ ]:
mask_inv = 255 - mask
thinned = cv.ximgproc.thinning(mask_inv)
ret = img.copy()
ret[thinned == 255] = [255, 0, 255]

isp.show_image_chain([mask, ret], ncols=2, 
                     titles=["Input (inverted mask)", 
                             "Skeletonization of the inverted mask"], 
                     suppress_info=True, figsize=(7, 12))


## **Voronoi tessellation and Delaunay triangulation** (advanced topic)

Voronoi tessellation and Delaunay triangulation are geometric constructions often used in image processing, computer vision, and computational geometry.

A **Voronoi tessellation** partitions a plane into regions based on the distance to a set of seed points. Each region contains all the points closer to its seed than to any other. This is useful for modeling influence zones or proximity. The Voronoi tessellation is conceptually related to the nearest neighbor rule – each Voronoi cell represents the region where its seed is the nearest neighbor to any point within that region.

A **Delaunay triangulation** connects points such that no point lies inside the circumcircle of any triangle created in this process. It is often used for mesh generation, interpolation, and feature extraction. The Delaunay triangulation and the Voronoi tessellation are dual operations. Each edge in the Delaunay triangulation corresponds to a shared boundary between two Voronoi regions.

In the following, let's operate with the centroids of the cell blobs we already have calculated.

In [ ]:
image_mask = cv.cvtColor(mask, cv.COLOR_GRAY2RGB)
    
# Compute the Voronoi diagram
height, width = mask.shape
subdiv = cv.Subdiv2D((0, 0, width, height))
for i in range(1, len(df)):
    x = int(df["centroid_x"][i])
    y = int(df["centroid_y"][i])
    subdiv.insert((x, y))

image_voronoi = image_mask.copy()
facets, centers = subdiv.getVoronoiFacetList([])
for i in range(len(facets)):
    if len(facets[i]) == 0:
        continue
    # Draw the facets
    #cv.fillConvexPoly(image_mask, np.array(facets[i], dtype=np.int32), [0, 255, 0])
    cv.polylines(image_voronoi, [np.array(facets[i], dtype=np.int32)], 
                 isClosed=True, color=[255, 0, 255], thickness=2)
    
image_delaunay = image_mask.copy()
triangle_list = subdiv.getTriangleList()
for i in range(len(triangle_list)):
    pt1 = (int(triangle_list[i][0]), int(triangle_list[i][1]))
    pt2 = (int(triangle_list[i][2]), int(triangle_list[i][3]))
    pt3 = (int(triangle_list[i][4]), int(triangle_list[i][5]))
    cv.line(image_delaunay, pt1, pt2, [255, 255, 0], 2)
    cv.line(image_delaunay, pt2, pt3, [255, 255, 0], 2)
    cv.line(image_delaunay, pt3, pt1, [255, 255, 0], 2)
    # Draw the centroids
    cv.circle(image_delaunay, (int(round(pt1[0])), int(round(pt1[1]))), 5, [255, 0, 0], -1)
    

for i in range(1, len(df)):
    x = int(df["centroid_x"][i])
    y = int(df["centroid_y"][i])
    cv.circle(image_voronoi, (x, y), 5, [255, 0, 0], -1)
    cv.circle(image_delaunay, (x, y), 5, [255, 0, 0], -1)
    
isp.show_image_chain(images=[mask, image_voronoi, image_delaunay], 
                     titles=["Input", 
                             "Voronoi tessellation", 
                             "Delaunay triangulation"],
                     suppress_info=True,
                     figsize=(9, 5))

Observe that the edges of the Voronoi tessellation are perpendicular to the corresponding edges of the Delaunay triangulation. What a beautiful result!

Voronoi diagrams and Delaunay triangulations are great for analyzing spatial relationships between points. Their ability to model proximity and connectivity makes them valuable in applications ranging from object detection to biological structure modeling.

<!---
## **Evaluation metrics**

**TODO**! Intersection over Union, Dice score
--->
